<a href="https://colab.research.google.com/github/AsutoshDalei/Deep-Neural-Networks/blob/master/Energy_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy Efficiency Dataset
We perform energy analysis using 12 different building shapes. The buildings differ with respect to the glazing area, the glazing area distribution, and the orientation, amongst other parameters. 
We simulate various settings as functions on the mentioned characteristics to obtain 768 building shapes. The dataset comprises 768 samples and 8 features, aiming to predict two real valued responses, and hence is a ***multi target regression problem***.

Importing the required packages

In [0]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import files
import io
uploaded=files.upload()

Saving energyeff.csv to energyeff.csv


We observe that the columns of the dataset are not named properly.

In [3]:
df=pd.read_csv(io.BytesIO(uploaded['energyeff.csv']))
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


As the first step to preprocessing, we give in columns names upon our convinience.

In [4]:
df.columns=['Relative Compactness','Surface Area','Wall Area','Roof Area',
            'Overall Height','Orientation','Glazing Area','Glazing Area Distribution'
            ,'Heating Load','Cooling Load']
df.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


We define the input and the multi targets for our regression problem.

In [0]:
inputs=df.drop(['Heating Load','Cooling Load'],axis=1)
targetsold=df.loc[:,['Heating Load','Cooling Load']]

We scale and shuffle the data.

In [0]:
lec=preprocessing.MinMaxScaler()
targets=shuffle(lec.fit_transform(targetsold))
inputs=shuffle(lec.fit_transform(inputs))

Declaring and defining the training, validating and testing dataset. This is later transformed into tesor friendly npz format.

In [0]:
nosamples=inputs.shape[0]
trainsamples=int(nosamples*0.8)
valsamples=int(nosamples*0.1)

trainip,trainop=inputs[:trainsamples,:],targets[:trainsamples,:]
validip,validop=inputs[trainsamples:trainsamples+valsamples,:],targets[trainsamples:trainsamples+valsamples,:]
testip,testop=inputs[trainsamples+valsamples:,:],targets[trainsamples+valsamples:,:]

In [0]:
np.savez('training',ip=trainip,op=trainop)
np.savez('validating',ip=validip,op=validop)
np.savez('testing',ip=testip,op=testop)

In [0]:
traindata=np.load('training.npz')
validata=np.load('validating.npz')
testdata=np.load('testing.npz')

Declaring a model.

In [0]:
model=tf.keras.Sequential([
                           tf.keras.layers.Dense(25,activation='relu'),
                           tf.keras.layers.Dense(25,activation='relu'),
                           tf.keras.layers.Dense(2,activation='linear')
                           ])

Here, we have the ability to modify the learning rate and the patience level of the *ADAM optimizer* and *EarlyStopping callback* respectively, along with batch size, loss and number of epochs. 

In [49]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error',optimizer=opt,metrics=['accuracy'])
cbk=tf.keras.callbacks.EarlyStopping(patience=2)
#model.fit(x=traindata['ip'],y=traindata['op'],epochs=100,validation_data=(validata['ip'],validata['op']),batch_size=50,callbacks=cbk)
model.fit(x=traindata['ip'],y=traindata['op'],epochs=100,validation_data=(validata['ip'],validata['op']),batch_size=25)

Epoch 1/100
25/25 [==============================] - 0s 5ms/step - loss: 0.1059 - accuracy: 0.6906 - val_loss: 0.0502 - val_accuracy: 0.9079
Epoch 2/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0727 - accuracy: 0.8046 - val_loss: 0.0529 - val_accuracy: 0.9079
Epoch 3/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0715 - accuracy: 0.8713 - val_loss: 0.0554 - val_accuracy: 0.9079
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0710 - accuracy: 0.8795 - val_loss: 0.0494 - val_accuracy: 0.9211
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0713 - accuracy: 0.8811 - val_loss: 0.0563 - val_accuracy: 0.9211
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0712 - accuracy: 0.8469 - val_loss: 0.0542 - val_accuracy: 0.9211
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0707 - accuracy: 0.8762 - val_loss: 0.0527 - val_accuracy: 0.9211
Epoch 8/100
2

Obtaining the final loss and accuracy of our model using the testing data.

In [50]:
loss,acc=model.evaluate(testdata['ip'],testdata['op'])
print('Loss:{0:.2f}   Accuracy:{1:.3f}'.format(loss,acc*100))

3/3 [==============================] - 0s 2ms/step - loss: 0.0714 - accuracy: 0.8462
Loss:0.07   Accuracy:84.615


Here, we try to predict the result when we provide in a sample of our own choice.

In [51]:
trydata=np.array([0.98,	514.5,	294.0,	110.25,	7.0,	2,	0.0,	0])
trydata=lec.fit_transform([trydata])
print(model.predict(trydata.reshape(1,8),batch_size=1))
tryout=model.predict(trydata.reshape(1,8),batch_size=1)

[[0.33518118 0.2656673 ]]


To be in a relatable form, we inverse transform out obtained data output.

In [52]:
lec1.fit(targetsold)
lec1.inverse_transform(tryout)

array([[18.44187 , 20.764227]], dtype=float32)